## BETTER WAY 17 - 인수를 순회할 때는 방어적으로 하자
## Item17 - Be Defensive When Iterating Over Arguments

파라미터로 객체의 리스트를 받는 함수에서 리스트를 여러 번 순회해야 할 때가 간혹 발생한다. 예를 들어 서울에 방문한 여행자 수를 분석하고 싶다고 하자. 데이터 집합은 각 구의 방문자 수(연도별 백만 명 단위)라고 하면, 각 구에서 전체 여행자 중 몇 퍼센트인지 구하는 코드는 아래와 같다.

In [1]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [2]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


이제 파일에서 데이터를 읽어와 위의 작업을 수행하는 제너레이터를 만들어 보자.

In [3]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [6]:
it = read_visits('./examples/numbers.txt')
percentages = normalize(it)  # <-- normalize 함수를 호출하면서 이터레이터가 사용됨
print(percentages)

[]


위의 결과를 보면 아무것도 출력 되지 않는다. 그 이유는 이터레이터 `it`이  결과를 한 번만 생성하기 때문이다. 아래의 코드를 보면 확실히 알 수 있다. 

In [8]:
it = read_visits('./examples/numbers.txt')
print(list(it))  # <- 여기서 사용됨
print(list(it))  # <- 빈값을 출력

[100, 11, 123, 122, 166, 188, 298, 300, 345, 432, 315, 462, 176, 542]
[]


이 문제를 해결하려면 이터레이터의 복사본을 리스트에 저장해야 한다. 

In [ ]:
def normalize_copy(numbers):
    numbers = list(numbers)
    total = sum(numbers)